<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'> Equipo Docente IIC2233 2023-2</font>
</p>

# Tabla de contenidos

1. [*QThreads* y señales](#QThreads-y-señales)
    1. [Explicación y solución](#Explicación-y-solución)
2. [*isAutoRepeat* en *KeyPressEvent* ](#isAutoRepeat-en-KeyPressEvent)
3. [Sonidos en PyQT](#Sonidos-en-PyQT)

## *QThreads* y señales

En los ejemplos anteriores, se muestra el uso de `Thread`, `QThread` y `QTimer`, que envían cambios a una ventana siempre mediante **señales**. Las señales no son la única forma de conseguir este comportamiento, pero si son las **más escalable para generar cambios** de la interfaz gráfica.

A continuación se muestra un ejemplo de `QThread` que modifica directamente la posición de *labels* dentro de una ventana. Se crean 100 *threads*, cada uno con un `QLabel` que tiene un `QPixmap` de diferente color. La posición evoluciona al pasar el tiempo y en cada cambio el *thread* cambia **directamente** la posición de la etiqueta mediante `label.move()`. 

Si ejecutas en tu computador este código verás que funciona, pero luego de decenas o centenas de *threads* creados, el programa **colapsa** o bien no se ve nada moviéndose. Este código se encuentra en el archivo `3-pyqt-miscelaneo-ejemplo_1.py`.

```python

from random import randint
from time import sleep

from PyQt6.QtCore import QThread
from PyQt6.QtGui import QPixmap, QColor
from PyQt6.QtWidgets import QLabel, QWidget, QApplication
import sys


class Cuadrado(QThread):
    identificador = 0

    def __init__(self, label, limite_x, limite_y):
        super().__init__()
        self.id = Cuadrado.identificador
        Cuadrado.identificador += 1

        # guardamos el label
        self.label = label

        # Seteamos la posición inicial y la guardamos para usarla como una property
        self._posicion = (0, 0)
        self.posicion = (randint(0, limite_x), randint(0, limite_y))

    @property
    def posicion(self):
        return self._posicion

    # Cada vez que se actualicé la posición,
    # se actualiza la posición de la etiqueta
    @posicion.setter
    def posicion(self, valor):
        self._posicion = valor
        self.label.move(*self.posicion)

    def run(self):
        while True:
            sleep(0.1)
            nuevo_x = self.posicion[0] + randint(-2, 2)
            nuevo_y = self.posicion[1] + randint(-2, 2)
            self.posicion = (nuevo_x, nuevo_y)


class MiVentana(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("No uso de señales")
        self.setGeometry(200, 200, 500, 500)

        # Definimos QLabel para el fondo de la ventana
        self.fondo = QLabel(self)
        self.fondo.setStyleSheet("background: orange")
        self.fondo.setGeometry(0, 0, 500, 500)

        self.cuadrados = []
        self.labels = {}

        for i in range(100):
            self.crear_cuadrado()

        self.show()

    def crear_cuadrado(self):
        # Creamos el label y se lo pasamos al Cuadrado
        label = QLabel(self)
        label.setGeometry(-50, -50, 50, 50)
        # Creamos un QPixmap de color aleatorio
        pixmap = QPixmap(50, 50)
        pixmap.fill(QColor(randint(20, 200), randint(20, 200), randint(20, 200)))
        label.setPixmap(pixmap)
        label.show()

        nuevo_cuadrado = Cuadrado(label, self.width(), self.height())
        self.cuadrados.append(nuevo_cuadrado)
        nuevo_cuadrado.start()

```

### Explicación y solución

Esto ocurre debido a que hay múltiples *threads* haciendo cambios directos en la interfaz casi simultáneamente, lo que da espacio para potenciales errores de concurrencia, que son clásicos en *threads*.

En cambio, al usar señales se evade de mejor forma este problema ya que se delega al manejo de eventos de PyQt la realización de los cambios en la interfaz. El siguiente código muestra una adaptación del código anterior pero utilizando señales. Al ejecutarlo, verás que se actualiza de mejor forma los Labels en la ventana y no se cae con los 100 *threads* iniciados. Este código se encuentra en el archivo `3-pyqt-miscelaneo-ejemplo_2.py`.

```python
from random import randint
from time import sleep

from PyQt6.QtCore import QThread, pyqtSignal
from PyQt6.QtGui import QPixmap, QColor
from PyQt6.QtWidgets import QLabel, QWidget, QApplication
import sys


class Cuadrado(QThread):
    identificador = 0

    def __init__(self, senal_mover, limite_x, limite_y):
        super().__init__()
        self.id = Cuadrado.identificador
        Cuadrado.identificador += 1

        # guardamos la señal
        self.senal_mover = senal_mover

        # Seteamos la posición inicial y la guardamos para usarla como una property
        self._posicion = (0, 0)
        self.posicion = (randint(0, limite_x), randint(0, limite_y))

    @property
    def posicion(self):
        return self._posicion

    # Cada vez que se actualicé la posición,
    # se actualiza la posición de la etiqueta
    @posicion.setter
    def posicion(self, valor):
        self._posicion = valor
        self.senal_mover.emit(self.id, *self.posicion)

    def run(self):
        while True:
            sleep(0.1)
            nuevo_x = self.posicion[0] + randint(-2, 2)
            nuevo_y = self.posicion[1] + randint(-2, 2)
            self.posicion = (nuevo_x, nuevo_y)


class MiVentana(QWidget):
    senal_mover = pyqtSignal(int, int, int)

    def __init__(self):
        super().__init__()
        self.setWindowTitle("Correcto uso de señales")
        self.setGeometry(200, 200, 500, 500)

        # Definimos QLabel para el fondo de la ventana
        self.fondo = QLabel(self)
        self.fondo.setStyleSheet("background: orange")
        self.fondo.setGeometry(0, 0, 500, 500)

        self.cuadrados = []
        self.labels = {}

        for i in range(100):
            self.crear_cuadrado()

        self.senal_mover.connect(self.mover)
        self.show()

    def crear_cuadrado(self):
        # Creamos el label y se lo pasamos al Cuadrado
        label = QLabel(self)
        label.setGeometry(-50, -50, 50, 50)
        # Creamos un QPixmap de color aleatorio
        pixmap = QPixmap(50, 50)
        pixmap.fill(QColor(randint(20, 200), randint(20, 200), randint(20, 200)))
        label.setPixmap(pixmap)
        label.show()

        nuevo_cuadrado = Cuadrado(self.senal_mover, self.width(), self.height())
        self.labels[nuevo_cuadrado.id] = label
        self.cuadrados.append(nuevo_cuadrado)
        nuevo_cuadrado.start()

    def mover(self, id, x, y):
        self.labels[id].move(x, y)

```

La conclusión de este experimento es que, en general, el uso de señales en conjunto a *theading* permite evadir potenciales problemas de concurrencia.

## *isAutoRepeat* en *KeyPressEvent* 

Anteriormente aprendimos del método `keyPressEvent` para detectar cuando una tecla del teclado es presionado. Una situación de interés de este método es cuando mantenemos presionado una tecla por mucho tiempo. Por defecto, cuando ocurre este suceso, el método es llamado múltiples veces, pero hay situaciones donde no queremos que se ejecute múltiples veces el método si es que mantengo presionado. 

Aquí es donde sale al rescate el método `isAutoRepeat()`. Este método retorna un booleano que indica si el evento gatillado fue producto de presionar por primera vez una tecla (`evento.isAutoRepeat() == False`) o es la repetición del evento por mantener presionada la tecla (`evento.isAutoRepeat() == True`).

A continuación veremos un ejemplo donde detectamos y contamos cuántas veces entramos en un `if` cuando presionamos la letra `A` o la `W`. La diferencia es que la letra `A` incluye una verificación de que `evento.isAutoRepeat()` sea `False`. Este código se encuentra en el archivo `3-pyqt-miscelaneo-ejemplo_3.py`.


```python

import sys
from PyQt6.QtCore import pyqtSignal, Qt
from PyQt6.QtWidgets import QApplication, QWidget, QLabel


class MiVentana(QWidget):
    def __init__(self):
        super().__init__()
        # Configuramos los widgets de la interfaz
        self.label_w = QLabel("Presiona la tecla W", self)
        self.label_w_contador = QLabel("Presionada 0 veces", self)
        self.contador_w = 0

        self.label_a = QLabel("Presiona la tecla A", self)
        self.label_a_contador = QLabel("Presionada 0 veces", self)
        self.contador_a = 0

        self.label_w.setGeometry(10, 10, 230, 30)
        self.label_w_contador.setGeometry(10, 40, 230, 30)
        self.label_a.setGeometry(10, 100, 230, 30)
        self.label_a_contador.setGeometry(10, 130, 230, 30)

        # Configuramos las propiedades de la ventana.
        self.setWindowTitle("Ejemplo KeyPressEvent")
        self.setGeometry(50, 50, 250, 200)
        self.show()

    def keyPressEvent(self, evento):
        if event.key() == Qt.Key.Key_W:
            self.contador_w += 1
            self.label_w_contador.setText(f"Presionada {self.contador_w} veces")

        if event.key() == Qt.Key.Key_A and not event.isAutoRepeat():
            self.contador_a += 1
            self.label_a_contador.setText(f"Presionada {self.contador_a} veces")

```

## Sonidos en PyQT

Cuando uno interactúa con una interfaz gráfica, no puede faltar su música de fondo o algún sonido cuando ocurre algún evento. Por esto, ahora veremos cómo podemos reproducir archivos de audio desde la interfaz. En esta ocasión, veremos 2 objetos de PyQT para reproducir sonido.

### `QMediaPlayer`

Este objeto nos permite reproducir archivos `.mp3`. Para configurar correctamente este objeto, se debe:

1. Instanciar el objeto: `self.media_player_mp3 = QMediaPlayer(self)`
2. Entregarle un objeto tipo QAudioOutput: `self.media_player_mp3.setAudioOutput(QAudioOutput(self))`. Esto es para que el reproductor entienda que va a desplegar un sonido.
3. Definir un objeto tipo `QURL` con el paths a nuestro sonido: `file_url = QUrl.fromLocalFile(join("sounds", "waku-waku.mp3"))`.
4. Entregarle la URL a nuestro reproductor: `self.media_player_mp3.setSource(file_url)`
5. Finalmente usar `play()` para reproducir el sonido.
    
**Importante**: por definición, el formato MP3 utiliza un algoritmo con pérdida para conseguir un menor tamaño de archivo. Esto puede ocasionar que al momento de reproducir un sonido con `QMediaPlayer`, salga el siguiente _warning_ en la consola: _Could not update timestamps for skipped samples_ o _Could not update timestamps for discarded samples._. Para efectos de este curso, no te preocupes por dicho _warning_.
    
    
### `QSoundEffect`

Este objeto nos permite reproducir archivos `.wav`. Para configurar correctamente este objeto, se debe: 

1. Instanciar el objeto: `self.media_player_wav = QSoundEffect(self)`
2. Definir un objeto tipo `QURL` con el paths a nuestro sonido: `file_url = QUrl.fromLocalFile(join("sounds", "see-you-again.wav"))`.
3. Entregarle la URL a nuestro reproductor: `self.media_player_wav.setSource(file_url)`
4. Finalmente usar `play()` para reproducir el sonido. También podemos usar `stop()` para detenerlo.

A continuación se muestra un ejemplo donde utilizamos ambos objetos para reproducir 2 sonidos distintos. Este código se encuentra en el archivo `3-pyqt-miscelaneo-ejemplo_4.py`.

```python
import sys
from PyQt6.QtCore import QUrl
from PyQt6.QtWidgets import QApplication, QWidget, QPushButton
from PyQt6.QtMultimedia import QMediaPlayer, QAudioOutput, QSoundEffect
from os.path import join


class MiVentana(QWidget):
    def __init__(self):
        super().__init__()

        # Configuramos los widgets de la interfaz
        self.boton_sorpresa = QPushButton("Wooow", self)
        self.boton_empezar = QPushButton("Empezar musica de fondo", self)
        self.boton_parar = QPushButton("Parar música de fondo", self)

        self.boton_sorpresa.setGeometry(10, 10, 230, 30)
        self.boton_empezar.setGeometry(10, 50, 230, 30)
        self.boton_parar.setGeometry(10, 90, 230, 30)

        self.boton_sorpresa.clicked.connect(self.empezar_sonido_sorpresa)
        self.boton_empezar.clicked.connect(self.empezar_musica_fondo)
        self.boton_parar.clicked.connect(self.parar_musica_fondo)

        # Configuramos las propiedades de la ventana.
        self.setWindowTitle("Ejemplo Sonidos")
        self.setGeometry(50, 50, 250, 200)

        # Opción MP3: QMediaPlayer junto a QAudioOutput
        # No te preocupes si te sale, en consola.
        #  "Could not update timestamps for skipped samples"
        self.media_player_mp3 = QMediaPlayer(self)
        self.media_player_mp3.setAudioOutput(QAudioOutput(self))
        file_url = QUrl.fromLocalFile(join("sounds", "waku-waku.mp3"))
        self.media_player_mp3.setSource(file_url)

        # Opción Wav: QSoundEffect
        self.media_player_wav = QSoundEffect(self)
        self.media_player_wav.setVolume(0.1) # Opcional
        file_url = QUrl.fromLocalFile(join("sounds", "see-you-again.wav"))
        self.media_player_wav.setSource(file_url)

        # Mostrar ventana
        self.show()

    def empezar_sonido_sorpresa(self):
        self.media_player_mp3.play()

    def empezar_musica_fondo(self):
        if not self.media_player_wav.isPlaying():
            self.media_player_wav.play()

    def parar_musica_fondo(self):
        if self.media_player_wav.isPlaying():
            self.media_player_wav.stop()
```